# Imports

In [1]:
# v4l2-ctl --list-devices
# lsusb

camera = "logitech_1"  # "FOSCAM" or "logitech_1"
cam_idx = 0

import cv2
import cv2.aruco as aruco
import numpy as np
import time
from util import select_cameras
import glob

aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_1000)
parameters = aruco.DetectorParameters()
last_look_homography_time = 0
last_compute_homography_time = 0
H = None

if camera=="FOSCAM":
    rtsp_url = "rtsp://admin:@192.168.0.115:554/videoMain"
    cap = cv2.VideoCapture(rtsp_url)

    def get_frame():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            return None
        return frame

if camera=="logitech_1":
    cap = cv2.VideoCapture(0)
    calibration_data = np.load("./camera_calibration/logitech_1/logitech_1.npz")
    mtx = calibration_data['mtx']
    dist = calibration_data['dist']

    def get_frame():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            return None
        return frame
    
    img = get_frame()
    h, w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# Functions

In [9]:
def camera_pose_from_homography(H,newcameramtx):

    num, Rs, Ts, Ns = cv2.decomposeHomographyMat(H, newcameramtx)
    pose = list()
    for i in range(num):
        pose_temp = -Rs[i].transpose()@Ts[i]
        pose_temp = pose_temp*24/pose_temp[2]
        pose.append(pose_temp)
        print(f"Solution {i}: {pose_temp.flatten()}")

    return pose[2]
    # Decompose the homography matrix to get rotation and translation
    # This is a simplified version and may not handle all cases
    '''
    H1 = H[:, 0]
    H2 = H[:, 1]
    H3 = np.cross(H1, H2)
    
    # Normalize the vectors
    norm1 = np.linalg.norm(H1)
    norm2 = np.linalg.norm(H2)
    tnorm = (norm1 + norm2) / 2.0
    
    r1 = H1 / norm1
    r2 = H2 / norm2
    r3 = H3 / np.linalg.norm(H3)
    
    R = np.column_stack((r1, r2, r3))
    
    # Ensure R is a proper rotation matrix using SVD
    U, _, Vt = np.linalg.svd(R)
    R = U @ Vt
    
    t = H[:, 2] / tnorm
    
    return R, t
    '''

# Camera Loop

In [ ]:

#cap = cv2.VideoCapture(cam_idx)
if not cap.isOpened():
    print(f"Could not open camera {rtsp_url}")

while True:

    frame = get_frame()
    now = time.time()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detector = aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, rejected = detector.detectMarkers(gray)

    # Every 1 second, try to compute homography from markers 9, 14, 19
    
    if now - last_look_homography_time > 1.0 and ids is not None:
        last_look_homography_time = now
        marker_ids = [9, 14, 19, 25]
        img_pts = []
        world_pts = []
        for i, marker_id in enumerate(marker_ids):
            idx = np.where(ids.flatten() == marker_id)[0]
            if len(idx) > 0:
                # Use the first corner of the marker as the reference point
                # We'll use the center for better stability
                c = corners[idx[0]][0]
                center = c.mean(axis=0)
                img_pts.append(center)
                if marker_id == 9:
                    world_pts.append([0, 0,0])
                elif marker_id == 14:
                    world_pts.append([12, 0,0])
                elif marker_id == 19:
                    world_pts.append([0, 12,0])
                elif marker_id == 25:
                    world_pts.append([12, 12,0])
        if len(img_pts) == 4:
            last_compute_homography_time = now
            img_pts = np.array(img_pts, dtype=np.float32)
            ud_img_pts = cv2.undistortPoints(np.expand_dims(img_pts, axis=1), newcameramtx, dist, P=newcameramtx)
            world_pts = np.array(world_pts, dtype=np.float32)
            # Add a fourth point for homography (simulate a rectangle)
            # Estimate the fourth world point (12,12)
            H, status = cv2.findHomography(ud_img_pts, world_pts)
            #camera_location_w= camera_pose_from_homography(np.linalg.inv(H),mtx)
            retval, rvec, tvec = cv2.solvePnP(	world_pts, img_pts, mtx, dist)	
            camera_location_w = -np.matrix(cv2.Rodrigues(rvec)[0]).T * np.matrix(tvec)

            if H is not None:
                print("[INFO] Homography matrix computed.")
            else:
                print("[WARN] Homography computation failed.")    

    # If homography is available, look for marker 25 and report its position
    if H is not None and ids is not None:
        idx25 = np.where(ids.flatten() == 25)[0]
        if len(idx25) > 0:
            c25 = corners[idx25[0]][0]
            center25 = c25.mean(axis=0)
            pt = np.array([[center25]], dtype=np.float32)
            world_pt = cv2.perspectiveTransform(pt, H)[0][0]
            cv2.putText(frame, f"25: ({world_pt[0]:.1f},{world_pt[1]:.1f})", tuple(center25.astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            print(f"[INFO] Marker 25 position: ({world_pt[0]:.2f}, {world_pt[1]:.2f})")


    # Draw and label all detected markers
    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        for i, corner in enumerate(corners):
            c = corner[0]
            center = c.mean(axis=0).astype(int)
            cv2.putText(frame, f"ID:{ids[i][0]}", tuple(center), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            pt1 = tuple(c[0].astype(int))
            pt2 = tuple(c[1].astype(int))
            cv2.arrowedLine(frame, pt1, pt2, (255,0,0), 2, tipLength=0.3)
            cv2.putText(frame, f"Last Homography:{last_compute_homography_time-now:.2f}", (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

        # Draw X and Y axes using marker centers if all three are found
        marker_centers = {}
        for marker_id in [9, 14, 19]:
            idx = np.where(ids.flatten() == marker_id)[0]
            if len(idx) > 0:
                c = corners[idx[0]][0]
                center = c.mean(axis=0).astype(int)
                marker_centers[marker_id] = center
                
        # Draw X axis: arrow from 9 to 14, label 'x'
        if 9 in marker_centers and 14 in marker_centers:
            cv2.arrowedLine(
                frame,
                tuple(marker_centers[9]),
                tuple(marker_centers[14]),
                (0, 0, 255), 3, tipLength=0.2
            )
            mid_x = ((marker_centers[9] + marker_centers[14]) // 2).astype(int)
            cv2.putText(frame, "x", tuple(mid_x), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Draw Y axis: arrow from 9 to 19, label 'y'
        if 9 in marker_centers and 19 in marker_centers:
            cv2.arrowedLine(
                frame,
                tuple(marker_centers[9]),
                tuple(marker_centers[19]),
                (0, 255, 255), 3, tipLength=0.2
            )
            mid_y = ((marker_centers[9] + marker_centers[19]) // 2).astype(int)
            cv2.putText(frame, "y", tuple(mid_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

            # Define a world-plane grid
            xmax, ymax = 10, 10  # 10x10 grid
            grid_world_points = []
            for y in range(ymax + 1):
                for x in range(xmax + 1):
                    grid_world_points.append([x, y])

            grid_world_points = np.array(grid_world_points, dtype=np.float32).reshape(-1, 1, 2)

            # Project world → image using homography
            grid_image_points = cv2.perspectiveTransform(grid_world_points, np.linalg.inv(H))

            # Draw grid lines
            for y in range(ymax + 1):
                # Horizontal lines
                pt_start = grid_image_points[y*(xmax+1)]
                pt_end = grid_image_points[y*(xmax+1) + xmax]
                cv2.line(frame,
                        tuple(pt_start[0].astype(int)),
                        tuple(pt_end[0].astype(int)),
                        (0, 255, 0), 1)

            for x in range(xmax + 1):
                # Vertical lines
                pt_start = grid_image_points[x]
                pt_end = grid_image_points[x + ymax*(xmax+1)]
                cv2.line(frame,
                        tuple(pt_start[0].astype(int)),
                        tuple(pt_end[0].astype(int)),
                        (0, 255, 0), 1)

    if H is not None:
        cv2.putText(frame, f"Camera Location:{camera_location_w}", (10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 1)

    cv2.imshow(f"Aruco Camera {cam_idx}", frame)
    key = cv2.waitKey(1)
    if key == 27 or key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



[INFO] Homography matrix computed.
[INFO] Marker 25 position: (11.98, 11.96)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.98, 11.96)
[INFO] Marker 25 position: (11.98, 11.96)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.98, 11.96)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.98, 11.98)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.98, 11.96)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.98, 11.96)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.97, 11.95)
[INFO] Marker 25 position: (11.98, 11.96)
[INFO] Marker 25 position: (11.98, 11.96)
[INFO] Marker 25 position: (11.98, 11.96)

KeyboardInterrupt: 

: 

In [ ]:
print(H)

[[ 6.71049117e-03  3.27555850e-01 -7.82604742e+01]
 [ 1.55272902e-01  5.55826785e-02 -3.15063241e+01]
 [ 7.85605784e-04  5.42625607e-03  1.00000000e+00]]


In [ ]:
cap.release()

In [ ]:
import cv2

camera_url = "http://ad@192.168.0.115:80/videoMain"  # Replace with your camera's URL
# rtsp://username:pwd@IP address:RTSP port/videoMain


while True:
    ret, frame = cap.read()

In [ ]:
print(H)
num, Rs, Ts, Ns = cv2.decomposeHomographyMat(H, np.eye(3))
print(f"Decomposed Homography into {num} solutions.")
for i in range(num):
    print(f"Solution {i+1}:")
    print("Rotation:\n", Rs[i])
    print("Translation:\n", Ts[i])
    print("Normal:\n", Ns[i])
    print()


[[ 6.71049117e-03  3.27555850e-01 -7.82604742e+01]
 [ 1.55272902e-01  5.55826785e-02 -3.15063241e+01]
 [ 7.85605784e-04  5.42625607e-03  1.00000000e+00]]
Decomposed Homography into 4 solutions.
Solution 1:
Rotation:
 [[-8.11078578e-02 -9.95640498e-01  4.60599017e-02]
 [-9.96694956e-01  8.12312076e-02  8.09538185e-04]
 [-4.54751014e-03 -4.58420112e-02 -9.98938352e-01]]
Translation:
 [[494.45351342]
 [199.07623414]
 [ -5.31969687]]
Normal:
 [[ 7.82819722e-05]
 [-2.17221901e-03]
 [ 9.99997638e-01]]

Solution 2:
Rotation:
 [[-8.11078578e-02 -9.95640498e-01  4.60599017e-02]
 [-9.96694956e-01  8.12312076e-02  8.09538185e-04]
 [-4.54751014e-03 -4.58420112e-02 -9.98938352e-01]]
Translation:
 [[-494.45351342]
 [-199.07623414]
 [   5.31969687]]
Normal:
 [[-7.82819722e-05]
 [ 2.17221901e-03]
 [-9.99997638e-01]]

Solution 3:
Rotation:
 [[ 0.74875531  0.65994575  0.06194431]
 [-0.66263686  0.74759303  0.0449116 ]
 [-0.01666991 -0.07467438  0.99706863]]
Translation:
 [[494.44462866]
 [199.03494965]


# Calibrate Camera

In [2]:
# Termination criteria for sub-pixel refinement
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points (e.g., for a 6x9 inner corner chessboard)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:6,0:9].T.reshape(-1,2)

# Arrays to store object points and image points from all images
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane

images = glob.glob('path/to/your/chessboard_images/*.jpg')

while True:
    img = get_frame()
    cv2.imshow('img', img)
    key = cv2.waitKey(1)
    if key == 27 or key == ord('q'):
        break
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (6,9), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners (optional)
        img = cv2.drawChessboardCorners(img, (6,9), corners2, ret)
        cv2.imshow('img', img)
        key = cv2.waitKey(1)
        if key == 27 or key == ord('q'):
            break
        cv2.waitKey(500)
    else:
        cv2.imshow('img', img)
        key = cv2.waitKey(1)
        if key == 27 or key == ord('q'):
            break

cv2.destroyAllWindows()

# Perform camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Save the calibration results (optional)
# np.savez('calibration_data.npz', mtx=mtx, dist=dist)

KeyboardInterrupt: 

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [ ]:
import os
folder_path = "./camera_calibration/logitech_1"

# Create the directory if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Define the full file path for the .npz file
filepath = os.path.join(folder_path, "logitech_1.npz")

# Save the matrices using numpy.savez
np.savez(filepath, mtx=mtx, dist=dist)

In [ ]:
# undistort
img = get_frame()
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))


dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imshow('calibresult.png', dst)
cv2.waitKey(1)
cv2.imshow('distorted', img)
cv2.waitKey(1)

-1

In [2]:
import cv2
import numpy as np

# Load your camera matrix (mtx) and distortion coefficients (dist)
# from the file saved during calibration.
# The path should match where you saved the .npz file.
try:
    calibration_data = np.load("./camera_calibration/logitech_1/logitech_1.npz")
    mtx = calibration_data['mtx']
    dist = calibration_data['dist']
except FileNotFoundError:
    print("Calibration file not found. Please run the calibration script first.")
    exit()


while True:
    img = get_frame()
    if img is None:
        continue
    # Get image dimensions
    h, w = img.shape[:2]

    # Get the new optimal camera matrix and a region of interest (ROI)
    # The '1' alpha value retains all original pixels, adding black borders if necessary.
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

    # Undistort the image
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # Crop the image to the valid region
    x, y, w_roi, h_roi = roi
    dst_cropped = dst[y:y+h_roi, x:x+w_roi]

    # Resize the cropped undistorted image to match the original size
    dst_resized = cv2.resize(dst_cropped, (w, h), interpolation=cv2.INTER_LINEAR)

    # Concatenate the two images horizontally
    # First, make sure both images have the same dimensions.
    comparison = np.hstack((img, dst_resized))

    # Display the result
    cv2.imshow('Distorted (Left) and Undistorted (Right)', comparison)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()
